In [1]:
!pip install emot
!pip install emoji
!pip install pyspellchecker
!pip install -U deep-translator
!pip install spacy -q
!pip install gensim -q
!python -m spacy download es_core_news_sm -q
!apt-get update
!apt-get install python-dev
!apt-get install -y hunspell-es
!apt-get install libhunspell-dev
!pip install hunspell

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 99.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64 

In [2]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 535 (delta 174), reused 130 (delta 94), pack-reused 269 (from 1)
Receiving objects: 100% (535/535), 172.47 KiB | 17.25 MiB/s, done.
Resolving deltas: 100% (276/276), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.8 MB/s eta 0:00:00
Installing RAPIDS remaining 24.10.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.7/567.7 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 831.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.5/915.5 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install emoji==2.13.0 flask-ngrok pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 32.2 MB/s eta 0:00:00
  Attempting uninstall: emoji
    Found existing installation: emoji 2.14.0
    Uninstalling emoji-2.14.0:
      Successfully uninstalled emoji-2.14.0


In [4]:
import joblib
import pandas as pd
from pydantic import BaseModel
import matplotlib.pyplot as plt
import re
import emoji
import emot
import spacy
import nltk
import gensim
import hunspell
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from deep_translator import GoogleTranslator
from urllib.request import urlopen
import cudf
import cuml

In [5]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [6]:
!pip install flask-cors

In [7]:
from pyngrok import ngrok
from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin

In [8]:
!ngrok config add-authtoken '2iM77czcs5Mfn1OmbTqRvxlLFCV_7B1GncpDXEbUEPyCU8kSc'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
app = Flask(__name__)

In [10]:
class MessageRequest(BaseModel):
    technique: str
    message: str
    algorithm: str

    def correct_spelling(self):
        hs = hunspell.HunSpell('/usr/share/hunspell/es_PE.dic', '/usr/share/hunspell/es_PE.aff')
        words = self.message.split()
        corrected_text = [
            hs.suggest(word)[0] if not hs.spell(word) and hs.suggest(word) else word
            for word in words
        ]
        self.message = ' '.join(corrected_text)
        return self

    def add_space_between_emojis(self):
        emojis = [char for char in self.message if char in emoji.EMOJI_DATA]
        for emoji_char in emojis:
            pattern = re.escape(emoji_char)
            self.message = re.sub(f'({pattern})', r' \1 ', self.message)
        return self


    def add_space_between_emoticons(self):
        emoticons = [char for char in self.message if char in emot.EMOTICONS_EMO.keys()]
        for emoticon in emoticons:
            pattern = re.escape(emoticon)
            self.message = re.sub(f'({pattern})', r' \1 ', self.message)
        return self

    def emoji_to_description(self):
        self.message = emoji.replace_emoji(self.message, replace=lambda chars, data_dict: GoogleTranslator(source='en', target='es')
        .translate(' '.join(data_dict.get('en', chars).split('_')).strip(':')))
        return self

    def emoticon_to_description(self):
        emot_obj = emot.core.emot()
        emoticon_info = emot_obj.emoticons(self.message)
        for _ in range(len(emoticon_info['value'])):
            emoticon_locations = emoticon_info['location'][0]
            emoticon_meanings = emoticon_info['mean'][0]
            emoticon_meanings = GoogleTranslator(source='en', target='es').translate(text=emoticon_meanings).lower()
            self.message = (
                self.message[:emoticon_locations[0]] +
                emoticon_meanings +
                self.message[emoticon_locations[1]:]
            )
            emoticon_info = emot_obj.emoticons(self.message)
        return self

    def to_lowercase(self):
        self.message = self.message.lower()
        return self

    def remove_tags(self):
        self.message = re.sub(r'#\w+', '', self.message)
        return self

    def remove_mentions(self):
        self.message = re.sub(r'@\w+', '', self.message)
        return self

    def remove_urls(self):
        self.message = re.sub(r'http\S+', '', self.message)
        return self

    def remove_numbers(self):
        self.message = re.sub(r'\d+', '', self.message)
        return self

    def remove_punctuation(self):
        self.message = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚüÜñÑ\s]', '', self.message)
        return self

    def remove_extra_spaces(self):
        self.message = re.sub(r'[\r\t\n]', ' ', self.message)
        self.message = re.sub(r'\s{2,}', ' ', self.message)
        return self

    def remove_repetitive_characters(self):
        allowed_words = {'rr', 'll', 'cc', 'ee', 'oo'}
        pattern = re.compile(r'(\w)\1+')

        def replacement(match):
            char = match.group(1)
            repeated = match.group(0)
            if char * 2 in allowed_words and len(repeated) > 2:
                return char * 2
            return char

        self.message = pattern.sub(replacement, self.message)
        return self

    def tokenize_apply_technique(self):
        tokens = word_tokenize(self.message, language='spanish')
        tweet_tokenized = [word for word in tokens]
        spanish_peruvian_dict_url = 'https://drive.google.com/uc?export=download&id=17J9JgVzdSeXKMk6hhsA8Zv2XCjIdMH84'
        spanish_peruvian_dict_df = pd.read_excel(spanish_peruvian_dict_url).dropna().reset_index(drop=True)
        spanish_peruvian_dict = dict(zip(spanish_peruvian_dict_df['jerga'], spanish_peruvian_dict_df['significado']))
        tweet_tokenized = [spanish_peruvian_dict.get(word, word) for word in tweet_tokenized]
        stopwords_es = set(stopwords.words('spanish'))
        tweet_tokenized = [word for word in tweet_tokenized if word.lower() not in stopwords_es]
        if self.technique == "lematización":
          nlp = spacy.load('es_core_news_sm')
          tweet_tokenized = [nlp(word)[0].lemma_ for word in tweet_tokenized]
          self.message = ' '.join(tweet_tokenized)
        else:
          stemmer = SnowballStemmer('spanish')
          self.message = ' '.join([stemmer.stem(word) for word in tweet_tokenized])
        return self


    def process_message(self):
        return (self.correct_spelling()
                .add_space_between_emojis()
                .add_space_between_emoticons()
                .emoji_to_description()
                .emoticon_to_description()
                .to_lowercase()
                .remove_tags()
                .remove_mentions()
                .remove_urls()
                .remove_numbers()
                .remove_punctuation()
                .remove_extra_spaces()
                .remove_repetitive_characters()
                .tokenize_apply_technique()
                .message)

In [11]:
def get_drive_url(id):
    return f'https://drive.google.com/uc?export=download&id={id}'

with app.app_context():
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('punkt_tab')
    nltk.download('wordnet')
    nltk.download('omw-1.4')

    global model_arbol_decision_tf_lematizacion
    global model_arbol_decision_tf_tallado
    global model_knn_tf_lematizacion
    global model_knn_tf_tallado
    global model_maquina_vectores_soporte_clasificador_tf_lematizacion
    global model_maquina_vectores_soporte_clasificador_tf_tallado
    global model_naive_bayes_clasificador_tf_lematizacion
    global model_naive_bayes_clasificador_tf_tallado
    global model_regresion_logistica_tf_lematizacion
    global model_regresion_logistica_tf_tallado
    global model_rf_tf_lematizacion
    global model_rf_tf_tallado
    global model_sgd_tf_lematizacion
    global model_sgd_tf_tallado
    global model_xg_tf_lematizacion
    global model_xg_tf_tallado

    model_arbol_decision_tf_lematizacion = joblib.load(urlopen(get_drive_url('1Pc-PM_OzIRvbt8fWSkvjYqooUgp4LY8m')))
    model_arbol_decision_tf_tallado = joblib.load(urlopen(get_drive_url('1VAZMdamHXfvIBJ3yrLYb21Gfxt61Kjzo')))
    model_knn_tf_lematizacion = joblib.load(urlopen(get_drive_url('1wr-RFfHDrM9Deykn1EpkbC-HnaLH-fQP')))
    model_knn_tf_tallado = joblib.load(urlopen(get_drive_url('1jKwPiWdsEMBnz6MZ2LlajKrVLOJaLLGN')))
    model_maquina_vectores_soporte_clasificador_tf_lematizacion = joblib.load(urlopen(get_drive_url('1GLpqFJwuuspHhze2wAMdWXyggE4uZdKh')))
    model_naive_bayes_clasificador_tf_lematizacion = joblib.load(urlopen(get_drive_url('13Rugb8U858qY9J7yeM8R74bKL_hNjP-A')))
    model_naive_bayes_clasificador_tf_tallado = joblib.load(urlopen(get_drive_url('1I-nMaTLi041Sfpx-fz-45xiIJ66zgHf6')))
    model_regresion_logistica_tf_lematizacion = joblib.load(urlopen(get_drive_url('1B273Q08Hm5dwlFjvdZhOI4aRG0oe3YuK')))
    model_regresion_logistica_tf_tallado = joblib.load(urlopen(get_drive_url('1j1b0bJ6pnttcP2RYzoODvsO7kfEW6u6t')))
    model_rf_tf_lematizacion = joblib.load(urlopen(get_drive_url('14sNsCStmiVAtJosoOHz664LrWdnCSpUz')))
    model_rf_tf_tallado = joblib.load(urlopen(get_drive_url('13TbFFEACF4ltFuNuxfQwYzNz9tvOjKuP')))
    model_sgd_tf_lematizacion = joblib.load(urlopen(get_drive_url('1Fv10hNR4OAynwrOU_ZuJyLqsgkBt5rTS')))
    model_sgd_tf_tallado = joblib.load(urlopen(get_drive_url('1N9dc-RlB-1C9qVs5fmne1G7h_Ml73L8k')))
    model_xg_tf_lematizacion = joblib.load(urlopen(get_drive_url('1DQKsNupl9jb1qx7m2KsgYe1ll5QGkaTU')))
    model_xg_tf_tallado = joblib.load(urlopen(get_drive_url('1_pmAFULr6LF_fkdw3Agr9HQfiUXj1Gew')))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [12]:
CORS(app, origins=["*"], methods=["POST", "OPTIONS"])
@app.route('/peruvian-ciberbullying/detection', methods=['POST', 'OPTIONS'])
@cross_origin(origins=["*"], methods=["POST", "OPTIONS"])
def detect_peruvian_ciberbullying():
    data = request.get_json()
    if not data:
        return jsonify({"error": "Body request is required."}), 400

    if 'technique' not in data or 'algorithm' not in data or 'message' not in data:
        return jsonify({"error": "Fields technique, algorithm and message are required."}), 400

    message = MessageRequest(**data)

    try:
        processed_message = message.process_message()
        model = None
        technique = message.technique
        algorithm = message.algorithm
        if technique == 'lematizacion' and algorithm == 'arbol_decision':
          model = model_arbol_decision_tf_lematizacion
        elif technique == 'tallado' and algorithm == 'arbol_decision':
          model = model_arbol_decision_tf_tallado
        elif technique == 'lematizacion' and algorithm == 'knn':
          model = model_knn_tf_lematizacion
        elif technique == 'tallado' and algorithm == 'knn':
          model = model_knn_tf_tallado
        elif technique == 'lematizacion' and algorithm == 'maquina_vectores_soporte':
          model = model_maquina_vectores_soporte_clasificador_tf_lematizacion
        elif technique == 'lematizacion' and algorithm == 'naive_bayes':
          model = model_naive_bayes_clasificador_tf_lematizacion
        elif technique == 'tallado' and algorithm == 'naive_bayes':
          model = model_naive_bayes_clasificador_tf_tallado
        elif technique == 'lematizacion' and algorithm == 'regresion_logistica':
          model = model_regresion_logistica_tf_lematizacion
        elif technique == 'tallado' and algorithm == 'regresion_logistica':
          model = model_regresion_logistica_tf_tallado
        elif technique == 'lematizacion' and algorithm == 'bosque_aleatorio':
          model = model_rf_tf_lematizacion
        elif technique == 'tallado' and algorithm == 'bosque_aleatorio':
          model = model_rf_tf_tallado
        elif technique == 'lematizacion' and algorithm == 'gradiente_descendente_estocástica':
          model = model_sgd_tf_lematizacion
        elif technique == 'tallado' and algorithm == 'gradiente_descendente_estocástica':
          model = model_sgd_tf_tallado
        elif technique == 'lematizacion' and algorithm == 'xgboost':
          model = model_xg_tf_lematizacion
        elif technique == 'tallado' and algorithm == 'xgboost':
          model = model_xg_tf_tallado
        else:
          model = model_rf_tf_lematizacion
        print(model.predict([processed_message]))
        prediction = int(model.predict([processed_message])[0])
        if prediction == 0:
            result = "No acoso"
        elif prediction == 1:
            result = "Hostigamiento directo"
        elif prediction == 2:
            result = "Discurso de odio"
        else:
            result = "Acoso sexual"
        return {"class": result}
    except Exception as e:
        return jsonify({"error": e}), 500

In [13]:
!pip install --upgrade pyngrok

In [14]:
ngrok.set_auth_token("2iM77czcs5Mfn1OmbTqRvxlLFCV_7B1GncpDXEbUEPyCU8kSc")
http_tunnel = ngrok.connect(5000, bind_tls=True)

print("Ngrok tunnel URL:", http_tunnel.public_url, )

Exception in thread Thread-10 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 139, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


Ngrok tunnel URL: https://b3e2-34-126-191-125.ngrok-free.app


In [15]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 02:41:17] "POST /peruvian-ciberbullying/detection HTTP/1.1" 200 -


[3]


INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 02:41:37] "POST /peruvian-ciberbullying/detection HTTP/1.1" 200 -


[0]


INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 02:42:11] "POST /peruvian-ciberbullying/detection HTTP/1.1" 200 -


[3]


INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 02:44:49] "OPTIONS /peruvian-ciberbullying/detection HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 02:44:57] "POST /peruvian-ciberbullying/detection HTTP/1.1" 200 -


[3]
